In [8]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
from sklearn import model_selection

In [9]:
path = "C:\\Users\\Ivand\\Repos\\banks\\data"

In [10]:
df_banks = pd.read_csv(path + "\\BanksDFPrepared.csv")
df_banks.drop(df_banks.columns[0], axis = 1, inplace = True)
df_bonds = pd.read_csv(path + "\\BondsPrepared.csv")
df_bonds.drop(df_bonds.columns[0], axis = 1, inplace = True)
df_bonds = df_bonds.rename({"cregnum" : "REGN"}, axis = 1)



In [11]:
df = df_bonds.merge(df_banks, how = "left", on = ["REGN", "Year", "Month"])
df_full = df.copy()

In [12]:
df.drop(["ISIN"], axis = 1, inplace = True)

In [13]:
df = pd.get_dummies(df, columns = ["Moodys", "ExpertRA"], drop_first = True)

In [14]:
df = df[~df.NetIncome.isnull()]
df.loc[df.Coupon.isnull(), "Coupon"] = 0
df.loc[df.Coupon_accum.isnull(), "Coupon_accum"] = 0

In [15]:
X_and_y_banks = df.REGN
X_and_y_banks = X_and_y_banks.unique()
X_and_y_banks_test = np.random.choice(X_and_y_banks, round(len(X_and_y_banks)/5))

X = df.drop(['Indicative_price', 'YTM_ind','RF_interpolated', 'G_spread_interpolated'], axis = 1)
y = df['G_spread_interpolated']

X_and_y_test_KNN = df[df.REGN.isin(X_and_y_banks_test)]
X_and_y_train_KNN = df[~df.REGN.isin(X_and_y_banks_test)]

X_test_KNN = X_and_y_test_KNN.drop(['G_spread_interpolated'], axis = 1)
X_train_KNN = X_and_y_train_KNN.drop(['G_spread_interpolated'], axis = 1)
y_test_KNN = X_and_y_test_KNN['G_spread_interpolated']
y_train_KNN = X_and_y_train_KNN['G_spread_interpolated']

X_train_KNN.reset_index(inplace=True, drop=True)
X_test_KNN.reset_index(inplace=True, drop=True)
y_train_KNN.reset_index(inplace=True, drop=True)
y_test_KNN.reset_index(inplace=True, drop=True)

In [16]:
class AbstractModule():
    def __init__(self):
        pass
    
    def set_params(self, **params):
        params = list(params.values())[0]
        for key in list(params.keys()):
            print(key)
            setattr(self, key, params[key])
        return self 
    
    def fit(self, X, y = None):
        pass
    
    def transform(self, X, y = None):
        pass
    


class Limiter(AbstractModule):
    def __init__(self, desired_columns, number_of_columns):
        super().__init__()
        self.desired_columns = desired_columns
        self.number_of_columns = number_of_columns
        

    
    def fit(self, X, y = None):
        self.X = X[self.desired_columns[:self.number_of_columns]]
        return self 
    
    def transform(self, X, y = None):
        return self.X
    

    
m = Limiter(['Month', 'Year', 'BA_spread', 'Indicative_price',
       'Turnover', 'Volume', 'Coupon', 'YTM_ind'], 11)
setparams = {"number_of_columns" : 1}
m.set_params(params = setparams)
m.fit(df)
m.transform(df)

number_of_columns


,Month
0,1
1,2
2,3
3,4
4,5
...,...
29792,9
29793,11
29794,12
29795,7


In [33]:
import pipeline_demo_module as pipe
from importlib import reload
pipe = reload(pipe)

In [18]:
pipe.algorithm_hint(True)

1. Define modules separately (separate_modules_hint)

<variable for module i> = <module function i> for i in modules

2. Define modules dictionary (dictionary_modules_hint)

<dictionary name> = {'<module name string i>' :
 <variable for module i> for i in modules}

3. Define parameters pipeline (pipeline_modules_hint)

<pipeline modules variable name> = {'<pipeline stage name j>' :
 hp.choice('<pipeline stage name j>', ['<module name string i>' for i in modules of stage j]) for j in stages}
'skip' as '<module name string i>' if you want to try doing nothing on the stage

4. Define hyperparameters dictionary (hyper_hint)

<modules hyperparameters list variable name> = {'<module name string i>__<parameter k name>' :
 [<parameter k value option m> for m in a list of possible values] for k in hyperparameters of module i, for i in modules}

5. Define loss function
6. Gather modules, parameters and loss into a dictionary (use gather_function)
7. Run optimizer


In [19]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler


In [20]:
estimator= LinearRegression(n_jobs = -1)
estimator.fit(X, y)
estimator.predict(X)

array([-0.08299059, -0.10577639, -0.09250348, ...,  0.01726247,
        0.05479509, -0.02209107])

In [21]:
best_linear_columns

NameError: name 'best_linear_columns' is not defined

In [23]:
best_knn_columns = ['MoscowSE',
 'NetInvestmentIncome',
 'BadBorrowing_Me',
 'CoreFunds_CBR',
 'Revaluation_CBR',
 'personal_page',
 'app',
 'foreign',
 'ownership_state',
 'form_public',
 'region_notmoscow',
 'NoMoodys',
 'Moodys_B2',
 'Moodys_B3',
 'Moodys_Ba1',
 'Moodys_Ba2',
 'Moodys_Ba3',
 'Moodys_Baa3',
 'Moodys_C',
 'Moodys_withdrawn',
 'ExpertRA_B',
 'ExpertRA_C',
 'ExpertRA_no',
 'ExpertRA_ruA',
 'ExpertRA_ruAA',
 'ExpertRA_ruAAA',
 'ExpertRA_ruB',
 'ExpertRA_ruBBB',
 'ExpertRA_ruC',
 'ExpertRA_withdrawn']

best_linear_columns = ['Month',
 'Turnover',
 'NumberOfInstances',
 'MoscowSE',
 'NetFeeIncome',
 'NetInvestmentIncome',
 'NetInterestIncome',
 'Assets',
 'Securities',
 'LendCorporate_Me',
 'ShareCapitalParticipation_CBR',
 'Revaluation_CBR',
 'RiskFree_norm',
 'CapitalAdequacy_norm',
 'LiquidityMonth_norm',
 'HighlyLiquidAssets_norm',
 'LiquidityDay_norm',
 'ShortCreditAssets_norm',
 'LiquidityLong_norm',
 'NoMoodys',
 'Moodys_B2',
 'Moodys_Ba2',
 'Moodys_Ba3',
 'Moodys_C',
 'ExpertRA_B',
 'ExpertRA_no',
 'ExpertRA_ruA',
 'ExpertRA_ruB',
 'ExpertRA_ruBB',
 'ExpertRA_ruC']

In [24]:


Limit_module = Limiter(desired_columns = best_linear_columns, 
                           number_of_columns = 10)

PCA_module = pipe.PCA_adj(
    n_components = 2,    # сколько оставить компонентов; по дефолту - все
    whiten = False,      # отключаем whitening - декорреляцию фичей
    svd_solver = "full", # детали SVD преобразования, за подробностями см. доки
)

kPCA_module = pipe.KernelPCA_adj(
    n_components = 8,  # сколько оставить компонентов; по дефолту - все
    kernel = "linear", # ядро. По дфеолту линейное. Можно сделать своё, но тогда его нужно предварительно вычислить отдельно,
                       # поставить kernel = "precomputed" и передать уже вычисленное ядро в качестве X
    degree = 3,        # степень полинома для некоторых типов ядер. Важный параметр для тьюнинга, но сильно напрягает процессор
    n_jobs = -1        # объект умеет быть многопоточным! -1 займет все ядра
)

Isomap_module = pipe.Isomap_adj(
    n_neighbors = 5, #количество соседей при вычислении KNN. Основной гиперпараметр, кстати (!!!)
    n_components = 2,  #сколько оставить компонент; по дефолту - 2
    path_method = "auto", #алгоритм, который вычисляет кратчайший путь. Варианты см. на странице функции. Этот подбирает сам.
    neighbors_algorithm = "auto", #алгоритм, который ищет соседей. Инстанс класса NearestNeighbours
    n_jobs = -1 #объект умеет быть многопоточным! -1 займет все ядра
)

UMAP_module = pipe.UMAP_adj(
    n_neighbors = 5,  # количество соседей при вычислении KNN. Основной гиперпараметр, кстати (!!!)
    n_components = 2, # сколько оставить компонентов; по дефолту - 2
    min_dist = 0.1    # минимальная дистанция, которую можно сохранять между точками в получающемся пространстве. Гиперпараметр. При увеличении начинает лучше улавливать общую структуру, но хуже - локальную
)

StScal = StandardScaler()


Basic = LinearRegression(n_jobs = -1)

KNN = KNeighborsRegressor(n_neighbors=5,
                           weights='uniform',
                           algorithm='auto',
                           leaf_size=30,
                           p=2,
                           metric='minkowski',
                           metric_params=None)
SVM = SVR(kernel='rbf',
          degree=3,
          gamma='scale',
          coef0=0.0,
          C=1.0, 
          epsilon=0.1,
          shrinking=True)

RF = RandomForestRegressor(n_estimators=100, 
                           criterion='squared_error',
                           max_depth=None,
                           min_samples_split=2,
                           min_samples_leaf=1,
                           min_weight_fraction_leaf=0.0,
                           max_features='auto',
                           max_leaf_nodes=None,
                           min_impurity_decrease=0.0,
                           bootstrap=True, 
                           oob_score=False, 
                           n_jobs=-1,
                           ccp_alpha=0.0)

In [25]:
modules = {
    "Limiter" : Limit_module,
    'PCA':         PCA_module,
    'kPCA':        kPCA_module,
 #   'Isomap':      Isomap_module,
    'UMAP':        UMAP_module,
    "StScal" : StScal,
    "Basic" : Basic,
    "KNN" : KNN,
    "SVR" : SVM,
    "RF" : RF
}

In [26]:
pipeline_modules = {
    "Restrict" : pipe.hp.choice("Restrict", ["skip", "Limiter"]),
    "Transform" : pipe.hp.choice("Transform", ["skip", "PCA", "kPCA", "UMAP"]),
    "Scale" : pipe.hp.choice("Scale", ["StScal", "skip"]),
    "Model" : pipe.hp.choice("Model", ["Basic", "KNN", "SVR", "RF"])
}

In [27]:
hyper_modules = {
    "Restrict_Limiter__desired_columns" : pipe.hp.choice("Limiter__desired_columns", [best_linear_columns, best_knn_columns]),
    "Limiter__number_of_columns" : pipe.hp.choice("Limiter__number_of_columns", [10, 15, 25, 30]),
    "Transform_PCA__n_components" : pipe.hp.choice("PCA__n_components", [10, 30, 50]),
    "Transform_PCA__whiten" : pipe.hp.choice("PCA__whiten", [True, False]),
    "Transform_kPCA__n_components" : pipe.hp.choice("kPCA__n_components", [10, 30, 50]),
    "Transform_kPCA__kernel" : pipe.hp.choice("kPCA__kernel", ["poly", "rbf"]),
    "Transform_kPCA__degree" : pipe.hp.choice("kPCA__degree", [1,2,3]),
    "Transform_UMAP__n_components" : pipe.hp.choice("UMAP__n_components", [10, 30, 50]),
    "Transform_UMAP__n_neighbours" : pipe.hp.choice("UMAP__n_neighbours", [2,3,5,10]),
    "Transform_UMAP__min_dist" : pipe.hp.choice("UMAP__min_dist", [0.05, 0.1,0.5, 1]),
    "Model_KNN__n_neighbours" : pipe.hp.choice("KNN__n_neighbours", [2, 5, 10]),
    "Model_KNN__weights" : pipe.hp.choice("KNN__weights", ["uniform", "distance"]),
    "Model_KNN__leaf_size" : pipe.hp.choice("KNN__leaf_size", [10, 20, 30]),
    "Model_KNN__p" : pipe.hp.choice("KNN__p", [1, 2, 1.5]),
    "Model_SVR__kernel" : pipe.hp.choice("SVR__kernel", ["poly", "rbf", "sigmoid"]),
    "Model_SVR__degree" : pipe.hp.choice("SVR__degree", [1,2,3]),
    "Model_SVR__C" : pipe.hp.choice("SVR__C", [0.1, 1, 10]),
    "Model_RF__criterion" : pipe.hp.choice("RF__criterion", ["squared_error", "poisson"]),
    "Model_RF__max_depth" : pipe.hp.choice("RF__max_depth", [None, 10, 20])
}

In [28]:
pipeline_modules = {
    "Restrict" : pipe.hp.choice("Restrict", ["skip", "Limiter"]),
    "Transform" : pipe.hp.choice("Transform", ["skip", "PCA"]),
    "Scale" : pipe.hp.choice("Scale", ["StScal", "skip"]),
    "Model" : "Basic"
}

In [29]:
def optimizer(X_train, y_train, pipe_para, modules, mode = "kfold", kfold_variable = None):
    hpoptimizer = PipeHPOpt(X_train, y_train, modules=modules, mode='kfold', n_folds = 5, seed=42, kfold_variable = None)
    lgb_opt, trials = hpoptimizer.process(space=pipe_para, trials=Trials(), algo=tpe.suggest, max_evals=10)
    return hpoptimizer

In [30]:
hasattr(LinearRegression, "fit_resample ")

False

In [47]:
import pipeline_demo_module as pipe
from importlib import reload
pipe = reload(pipe)


loss = lambda y, pred: mean_squared_error(y, pred)
# gathered_pipeline = pipe.gather_function(pipeline_modules, hyper_modules, loss)
# best_model = pipe.optimizer(X, y, gathered_pipeline, modules = modules, mode = "kfold_variable", kfold_variable = "REGN")

In [48]:
hpoptimizer = pipe.PipeHPOpt(X, y, modules=modules, task = "regression", pipe_para = gathered_pipeline, mode='kfold', n_folds = 5, seed=42, kfold_variable = None)
lgb_opt, trials = hpoptimizer.process(space=gathered_pipeline, trials=pipe.Trials(), algo=pipe.tpe.suggest, max_evals=10)

  0%|                                                                           | 0/10 [00:00<?, ?trial/s, best loss=?]

job exception: All intermediate steps of the chain should be estimators that implement fit and transform or fit_resample (but not both) or be a string 'passthrough' 'LinearRegression(n_jobs=-1)' (type <class 'sklearn.linear_model._base.LinearRegression'>) doesn't)



  0%|                                                                           | 0/10 [00:00<?, ?trial/s, best loss=?]


In [195]:
hpoptimizer

In [184]:

lgbm_mdl = LGBMClassifier(
    num_leaves = 10,
    learning_rate = .1,
    reg_alpha = 8,
    reg_lambda = 8,
    random_state = seed
)


NameError: name 'LGBMClassifier' is not defined

In [146]:
import os
os.system("rundll32.exe powrprof.dll,SetSuspendState 0,1,0")

0